# [CSC8101] Engineering for AI - 2024 Spark Coursework

### Kaustubh Kulkarni - 230195431

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html
  - actions to get around the lazy execution of spark: 
  https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/mutazb999/CSC8101-lab-and-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/parquet/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/delta/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'parquet'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
trips.take(1)


    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    


[Row(index=0, VendorID=1.0, tpep_pickup_datetime='2021-07-01 00:08:51', tpep_dropoff_datetime='2021-07-01 00:13:05', passenger_count=1.0, trip_distance=0.8, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=90, DOLocationID=68, payment_type=1.0, fare_amount=5.0, extra=3.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=8.8, congestion_surcharge=2.5, cab_type='Y', lpep_pickup_datetime=None, lpep_dropoff_datetime=None, ehail_fee=None, trip_type=None)]

In [0]:
print_count(trips)

Row count: 2,898,033


In [0]:
# dataset schemas
trips.printSchema()

root
 |-- index: long (nullable = true)
 |-- VendorID: double (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- cab_type: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |--

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3


In [0]:
zone_names.printSchema()

root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [0]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.483 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
    # input: trips dataset
    relevant_columns = ['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']
    df = df[relevant_columns]
    filtered_df = df[(df['trip_distance'] > 0 ) | ((df['passenger_count'] > 0) & (df['total_amount'] > 0))]
    return filtered_df

In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)


print_count(trips_11)

## uncomment only for smaller datasets
display(trips_11.take(10))

Row count: 2,888,435


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3
114,90,1.6,1.0,12.8
90,144,1.8,1.0,13.3
114,48,2.0,1.0,14.75
48,152,5.7,1.0,22.3
234,148,1.8,1.0,14.75


In [0]:
# Your solution implementation to task 1.2 goes HERE
def MAD(df):

    total_amount_median = df.agg(pf.percentile_approx("total_amount", 0.5)).collect()[0][0] # median of total_amount column (single value)
    trip_distance_median = df.agg(pf.percentile_approx("trip_distance", 0.5)).collect()[0][0] # median of trip_distance column (single value)
    total_amount_MAD = 1.438*df.agg(pf.percentile_approx(pf.abs(df.total_amount - total_amount_median), 0.5)).collect()[0][0] # MAD value of total amount calculated using the formula given above (single value)
    trip_distance_MAD = 1.438*df.agg(pf.percentile_approx(pf.abs(df.trip_distance - trip_distance_median), 0.5)).collect()[0][0] # MAD value of trip distance calculated using the formula given above (single value)
    
    return total_amount_median, total_amount_MAD, trip_distance_median, trip_distance_MAD

def z_score(df, total_amount_median, total_amount_MAD, trip_distance_median, trip_distance_MAD):
    
    z_total_amount = (df.total_amount - total_amount_median)/total_amount_MAD # Column with z score calculated based on total amount
    z_trip_distance = (df.trip_distance - trip_distance_median)/trip_distance_MAD # Column with z score calculated based on trip distance
    
    return z_total_amount, z_trip_distance


def t12_remove_outliers(df):

    total_amount_median, total_amount_MAD, trip_distance_median, trip_distance_MAD = MAD(df) # calling MAD function created above
    z_total_amount, z_trip_distance = z_score(df, total_amount_median, total_amount_MAD, trip_distance_median, trip_distance_MAD) # calling z_score function created above
    df = df.withColumn('z_total_amount', z_total_amount) # renaming the column
    df = df.withColumn('z_trip_distance', z_trip_distance) # renaming the column
    df = df.filter(pf.abs(df.z_total_amount) <= 3.5) # removing the rows which are having z_score more than 3.5
    df = df.filter(pf.abs(df.z_trip_distance) <= 3.5) # removing the rows which are having z_score more than 3.5

    return df

In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)

print_count(trips_12)
display(trips_12.take(10))

Row count: 2,510,877


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount,z_total_amount,z_trip_distance
90,68,0.8,1.0,8.8,-1.0585870956604446,-0.7886096095665516
113,90,0.9,1.0,8.8,-1.0585870956604446,-0.7169178268786832
88,232,2.8,1.0,13.8,-0.2442893297677949,0.6452260441908149
79,249,1.4,1.0,12.3,-0.4885786595355898,-0.35845891343934166
142,238,2.0,0.0,12.3,-0.4885786595355898,0.0716917826878684
114,90,1.6,1.0,12.8,-0.40714888294632484,-0.21507534806360487
90,144,1.8,1.0,13.3,-0.32571910635705986,-0.07169178268786823
114,48,2.0,1.0,14.75,-0.08957275424819158,0.0716917826878684
48,152,5.7,1.0,22.3,1.1400168722497095,2.724287742138997
234,148,1.8,1.0,14.75,-0.08957275424819158,-0.07169178268786823


## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitability = total_amount / trip_distance`.

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
# Your solution implementation to task 2.1 goes HERE
def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    zones_df = zones_df.withColumnRenamed("LocationID", "PULocationID") \
                       .withColumnRenamed("Borough", "PU_Borough") \
                       .withColumnRenamed("Zone", "PU_Zone") \
                       .withColumnRenamed("service_zone", "PU_service_zone") 
                       
    
    # Join the trips and zone_names datasets for pickup location
    trips_with_pickup_zone = df.join(zones_df, on="PULocationID", how="left")
    
    # Rename the columns in the zone_names DataFrame again for joining with dropoff location
    zones_df = zones_df.withColumnRenamed("PULocationID", "DOLocationID") \
                       .withColumnRenamed("PU_Borough", "DO_Borough") \
                       .withColumnRenamed("PU_Zone", "DO_Zone") \
                       .withColumnRenamed("PU_service_zone", "DO_service_zone")
    
    # Join the trips and zone_names datasets for dropoff location
    trips_with_zones = trips_with_pickup_zone.join(zones_df, on="DOLocationID", how="left")
    
    return trips_with_zones

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
display(trips_21.take(10))

Row count: 2,510,877


DOLocationID,PULocationID,trip_distance,passenger_count,total_amount,z_total_amount,z_trip_distance,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone
68,90,0.8,1.0,8.8,-1.0585870956604446,-0.7886096095665516,Manhattan,Flatiron,Yellow Zone,Manhattan,East Chelsea,Yellow Zone
90,113,0.9,1.0,8.8,-1.0585870956604446,-0.7169178268786832,Manhattan,Greenwich Village North,Yellow Zone,Manhattan,Flatiron,Yellow Zone
232,88,2.8,1.0,13.8,-0.2442893297677949,0.6452260441908149,Manhattan,Financial District South,Yellow Zone,Manhattan,Two Bridges/Seward Park,Yellow Zone
249,79,1.4,1.0,12.3,-0.4885786595355898,-0.35845891343934166,Manhattan,East Village,Yellow Zone,Manhattan,West Village,Yellow Zone
238,142,2.0,0.0,12.3,-0.4885786595355898,0.0716917826878684,Manhattan,Lincoln Square East,Yellow Zone,Manhattan,Upper West Side North,Yellow Zone
90,114,1.6,1.0,12.8,-0.40714888294632484,-0.21507534806360487,Manhattan,Greenwich Village South,Yellow Zone,Manhattan,Flatiron,Yellow Zone
144,90,1.8,1.0,13.3,-0.32571910635705986,-0.07169178268786823,Manhattan,Flatiron,Yellow Zone,Manhattan,Little Italy/NoLiTa,Yellow Zone
48,114,2.0,1.0,14.75,-0.08957275424819158,0.0716917826878684,Manhattan,Greenwich Village South,Yellow Zone,Manhattan,Clinton East,Yellow Zone
152,48,5.7,1.0,22.3,1.1400168722497095,2.724287742138997,Manhattan,Clinton East,Yellow Zone,Manhattan,Manhattanville,Boro Zone
148,234,1.8,1.0,14.75,-0.08957275424819158,-0.07169178268786823,Manhattan,Union Sq,Yellow Zone,Manhattan,Lower East Side,Yellow Zone


In [0]:
# Your solution implementation to task 2.2 goes HERE
from pyspark.sql.functions import col

def t22_calc_profit(df):
    # input: output of task 2.1
    df = df.withColumn("unit_profitability", col("total_amount") / col("trip_distance"))
    return df

In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)

print_count(trips_22)
display(trips_22.take(10))

Row count: 2,510,877


DOLocationID,PULocationID,trip_distance,passenger_count,total_amount,z_total_amount,z_trip_distance,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone,unit_profitability
68,90,0.8,1.0,8.8,-1.0585870956604446,-0.7886096095665516,Manhattan,Flatiron,Yellow Zone,Manhattan,East Chelsea,Yellow Zone,11.0
90,113,0.9,1.0,8.8,-1.0585870956604446,-0.7169178268786832,Manhattan,Greenwich Village North,Yellow Zone,Manhattan,Flatiron,Yellow Zone,9.777777777777779
232,88,2.8,1.0,13.8,-0.2442893297677949,0.6452260441908149,Manhattan,Financial District South,Yellow Zone,Manhattan,Two Bridges/Seward Park,Yellow Zone,4.928571428571429
249,79,1.4,1.0,12.3,-0.4885786595355898,-0.35845891343934166,Manhattan,East Village,Yellow Zone,Manhattan,West Village,Yellow Zone,8.785714285714286
238,142,2.0,0.0,12.3,-0.4885786595355898,0.0716917826878684,Manhattan,Lincoln Square East,Yellow Zone,Manhattan,Upper West Side North,Yellow Zone,6.15
90,114,1.6,1.0,12.8,-0.40714888294632484,-0.21507534806360487,Manhattan,Greenwich Village South,Yellow Zone,Manhattan,Flatiron,Yellow Zone,8.0
144,90,1.8,1.0,13.3,-0.32571910635705986,-0.07169178268786823,Manhattan,Flatiron,Yellow Zone,Manhattan,Little Italy/NoLiTa,Yellow Zone,7.388888888888889
48,114,2.0,1.0,14.75,-0.08957275424819158,0.0716917826878684,Manhattan,Greenwich Village South,Yellow Zone,Manhattan,Clinton East,Yellow Zone,7.375
152,48,5.7,1.0,22.3,1.1400168722497095,2.724287742138997,Manhattan,Clinton East,Yellow Zone,Manhattan,Manhattanville,Boro Zone,3.912280701754386
148,234,1.8,1.0,14.75,-0.08957275424819158,-0.07169178268786823,Manhattan,Union Sq,Yellow Zone,Manhattan,Lower East Side,Yellow Zone,8.194444444444445


## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z1 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitability)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
## Your solution to task 3.1 goes HERE
from pyspark.sql.functions import avg, sum, count, mean

def t31_summarise_trips(df):
    # input: output of task 2.2
    # Aggregate data for interzonal travel
    df = df.groupBy("PULocationID", "DOLocationID") \
                                 .agg(mean("unit_profitability").alias("average_unit_profit"),
                                      count("trip_distance").alias("trips_count"),
                                      sum("passenger_count").alias("total_passengers"))
    return df

In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
display(graph.take(10))

Row count: 15,617


PULocationID,DOLocationID,average_unit_profit,trips_count,total_passengers
17,26,5.862292456092071,5,3.0
228,26,7.41471058428216,10,4.0
108,26,6.236111111111111,1,1.0
142,26,null,1,1.0
25,26,4.722645989088705,3,4.0
21,29,7.742119414102729,10,2.0
97,26,4.504891363110619,9,10.0
22,26,15.249447857883409,33,3.0
76,26,7.056962025316456,3,2.0
76,29,null,2,2.0


In [0]:
# Your solution to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    df = df.join(zones_df.select("LocationID","Zone"),df['PULocationID'] == zones_df['LocationID'], how='left')
    df = df.withColumnRenamed('Zone', 'start')
    df = df.drop("LocationID")
    df = df.join(zones_df.select("LocationID","Zone"),df['DOLocationID'] == zones_df['LocationID'], how='left')
    df = df.withColumnRenamed('Zone', 'stop')
    df = df.drop("LocationID")
    return df

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    top10_trips = df_zones.orderBy("trips_count", ascending=False).limit(10)
    return top10_trips

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    top10_profit = df_zones.orderBy("average_unit_profit", ascending=False).limit(10)
    return top10_profit

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    top10_passenger = df_zones.orderBy("total_passengers", ascending=False).limit(10)
    return top10_passenger

In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_trips)

PULocationID,DOLocationID,average_unit_profit,trips_count,total_passengers,start,stop
237,236,11.57441123089697,15276,20893.0,Upper East Side South,Upper East Side North
236,237,11.865007247086426,13326,18567.0,Upper East Side North,Upper East Side South
237,237,19.048286369237072,11599,16351.0,Upper East Side South,Upper East Side South
236,236,20.41364428546659,9952,13906.0,Upper East Side North,Upper East Side North
264,264,15.08137084742967,8265,12601.0,NV,NV
237,161,11.717716194174908,7404,10295.0,Upper East Side South,Midtown Center
161,237,11.946247435087487,6824,9708.0,Midtown Center,Upper East Side South
237,162,12.571280048234033,6700,9456.0,Upper East Side South,Midtown East
239,142,13.17454084160379,6536,9292.0,Upper West Side South,Lincoln Square East
142,239,12.228503826579944,6446,9263.0,Lincoln Square East,Upper West Side South


In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
display(top10_profit)

PULocationID,DOLocationID,average_unit_profit,trips_count,total_passengers,start,stop
15,203,3409.0000000000005,1,null,Bay Terrace/Fort Totten,Rosedale
10,264,2736.0,10,12.0,Baisley Park,NV
190,264,2076.0,2,1.0,Prospect Park,NV
241,264,1530.0,1,1.0,Van Cortlandt Village,NV
89,264,1518.0,4,4.0,Flatbush/Ditmas Park,NV
25,264,988.4120569280344,4,6.0,Boerum Hill,NV
168,264,948.0,3,3.0,Mott Haven/Port Morris,NV
190,190,866.6666666666666,2,1.0,Prospect Park,Prospect Park
81,81,566.0705091732574,12,9.0,Eastchester,Eastchester
192,192,554.8688492063492,4,7.0,Queensboro Hill,Queensboro Hill


In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_passenger)

PULocationID,DOLocationID,average_unit_profit,trips_count,total_passengers,start,stop
237,236,11.57441123089697,15276,20893.0,Upper East Side South,Upper East Side North
236,237,11.865007247086426,13326,18567.0,Upper East Side North,Upper East Side South
237,237,19.048286369237072,11599,16351.0,Upper East Side South,Upper East Side South
236,236,20.41364428546659,9952,13906.0,Upper East Side North,Upper East Side North
264,264,15.08137084742967,8265,12601.0,NV,NV
237,161,11.717716194174908,7404,10295.0,Upper East Side South,Midtown Center
161,237,11.946247435087487,6824,9708.0,Midtown Center,Upper East Side South
237,162,12.571280048234033,6700,9456.0,Upper East Side South,Midtown East
239,142,13.17454084160379,6536,9292.0,Upper West Side South,Lincoln Square East
142,239,12.228503826579944,6446,9263.0,Lincoln Square East,Upper West Side South


## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'parquet'
WITH_TASK_12 = True

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)


    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    


In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

[DataFrame[PULocationID: bigint, DOLocationID: bigint, average_unit_profit: double, trips_count: bigint, total_passengers: double, start: string, stop: string],
 DataFrame[PULocationID: bigint, DOLocationID: bigint, average_unit_profit: double, trips_count: bigint, total_passengers: double, start: string, stop: string],
 DataFrame[PULocationID: bigint, DOLocationID: bigint, average_unit_profit: double, trips_count: bigint, total_passengers: double, start: string, stop: string]]

_Table 1. Pipeline performance for `parquet` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2.89 |  15.57 |  41.95 |  90.44 |  132.39 |
| execution time   (w/o 1.2)  | 0.36 | 0.22 | 0.28 | 0.31 | 0.43 |
| execution time              | 1.69 | 5.18 | 14.07 | 35.24 | 44.59 |
| sec / 1M records (w/o 1.2)  | 0.12  | 0.014  | 0.006  | 0.003  | 0.003  |
| sec / 1M records            | 0.58  | 0.332  | 0.335  | 0.38  | 0.336  |

_Table 2. Pipeline performance for `delta` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  000 |  000 |  000 |  000 |  000 |
| execution time   (w/o 1.2)  | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 |
| execution time              | 0.00 | 0.00 | 0.00 | 0.00 | 0.00 |
| sec / 1M records (w/o 1.2)  | 0.0  | 0.0  | 0.0  | 0.0  | 0.0  |
| sec / 1M records            | 0.0  | 0.0  | 0.0  | 0.0  | 0.0  |